# Day 10: Cathode-Ray Tube

_Find the signal strength during the 20th, 60th, 100th, 140th, 180th, and 220th cycles. What is the sum of these six signal strengths?_

Simulate executing the program given in the input.

Signal strength = cycle number * register value

Register value `X` starts at 1 and changes with the `addx` instruction

Two instructions:

`addx V`

* takes two cycles
* adds integer `V` to `X`
* new value shows up in `X` at the end of the second cycle

`noop`

* takes one cycle
* does nothing

In [1]:
from dataclasses import dataclass
from dataclasses import field
from typing import Dict
from typing import List
from typing import Optional
from typing import Callable
from enum import Enum
from enum import auto


In [2]:
class OpCode(Enum):
  ADDX = auto()
  NOOP = auto()

@dataclass
class Instruction:
  op_code : OpCode
  operand : Optional[int] = None

@dataclass
class Program:
  instructions : List[Instruction]

def instruction_cycles(instruction : Instruction) -> int:
  """ Gives the number of cycles required to execute `instruction`. """
  return {
    OpCode.ADDX: 2,
    OpCode.NOOP: 1
  }[instruction.op_code]


In [3]:
@dataclass
class Cpu:
  program : Program
  register : int = field(init=False, default=1)
  cycle_counter: int = field(init=False, default=0)
  program_counter: int = field(init=False, default=0)
  watches: Dict[int, List[Callable]] = field(init=False, default_factory=dict)

  def add_watch(self, cycle : int, watch : Callable):
    """ Add a function to call after `cycle` cycles."""
    if cycle not in self.watches.keys():
      self.watches[cycle] = []
    self.watches[cycle].append(watch)
  
  def execute_instruction(self, instruction : Instruction):
    if instruction.op_code == OpCode.ADDX:
      if not instruction.operand:
        raise Exception('ADDX requires an int operand.')
      self.register += instruction.operand
  
  def run(self, state: Dict[int, int]):
    current_instruction = self.program.instructions[self.program_counter]
    current_instruction_completion_cycle = instruction_cycles(current_instruction)
    while True:
      # Watches are called _during_ the cycle, not after.
      if self.cycle_counter in self.watches.keys():
        for w in self.watches[self.cycle_counter]:
          w(self, state)
      if self.cycle_counter == current_instruction_completion_cycle:
        self.execute_instruction(current_instruction)
        self.program_counter += 1
        if self.program_counter >= len(self.program.instructions):
          break
        current_instruction = self.program.instructions[self.program_counter]
        current_instruction_completion_cycle += instruction_cycles(current_instruction)
      self.cycle_counter += 1

In [4]:
def load_data(filename : str) -> Program:
  program = Program([])
  with open(filename) as f:
    for line in f.readlines():
      line = line.strip()
      if len(line) == 0 or line[0] == '#':
        continue
      match line.split():
        case ['noop']:
          program.instructions.append(Instruction(OpCode.NOOP))
        case ['addx', v]:
          program.instructions.append(Instruction(OpCode.ADDX, operand=int(v)))
        case _:
          raise Exception(f'Unknown instruction {line}')
    return program

In [5]:
def solver(filename : str):
  inspect_cycles = [20, 60, 100, 140, 180, 220]
  def save_signal_strength(cpu: Cpu, state: Dict[int, int]):
    state[cpu.cycle_counter] = cpu.cycle_counter * cpu.register
  program = load_data(filename)
  cpu = Cpu(program)
  for c in inspect_cycles:
    cpu.add_watch(c, save_signal_strength)
  state = dict()
  cpu.run(state)
  print(sum(state.values()))

In [6]:
solver('input.txt')

IndexError: list index out of range